In [1]:
from typing import Any
import numpy as np
from time import time

import pandas as pd
import scipy
from tqdm import tqdm

from common.evaluate import evaluate_pose_error_J3d_P2d, mmd_J3d_J3d
from paik.solver import NSF, PAIK, Solver, get_solver
from sklearn.cluster import DBSCAN
from common.config import Config_IKP
from common.display import save_ikp
from scipy.spatial.distance import cdist


import os

from ikflow.model import IkflowModelParameters
from ikflow.ikflow_solver import IKFlowSolver
import jrl.robots as jrlib
import paik.klampt_robot as chlib

import torch
from ikflow.training.lt_model import IkfLitModel
from experiments_ikflow import get_ikflow_solver

# set the same random seed for reproducibility
np.random.seed(0)
torch.manual_seed(0)

DEFAULT_WEIGHTS_PATH = {
    'atlas_arm': '/home/luca/ikflow/training_logs/atlas_arm--Sep.11.2024_07:52:PM/ikflow-checkpoint-step=90000.ckpt',
    'atlas_waist_arm': '/home/luca/ikflow/training_logs/atlas_waist_arm--Sep.10.2024_11:3:PM/ikflow-checkpoint-step=270000.ckpt',
    'baxter_arm': '/home/luca/ikflow/training_logs/baxter--Sep.12.2024_09:27:AM/ikflow-checkpoint-step=960000.ckpt',
}

ikflow/config.py | Using device: 'cuda:0'


In [2]:
robot_name = 'panda'
nsf = get_solver(arch_name="nsf", robot_name=robot_name, load=True, work_dir='/home/luca/paik')
transforms = nsf._solver.transforms
base = nsf._solver.base

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [3]:
nsf.random_ikp(num_poses=10, num_sols=100)

input z.shape: torch.Size([100, 10, 7]).
                l2          ang
count  1000.000000  1000.000000
mean      0.003696     1.478805
std       0.002848     3.012355
min       0.000198     0.065961
25%       0.002012     0.506973
50%       0.002878     0.791711
75%       0.004526     1.335286
max       0.027557    50.243552
  l2 (mm)    ang (deg)    inference_time (ms)
---------  -----------  ---------------------
      3.7         1.48                     34


(0.0036955545727540393, 0.025810013919472505, 0.034)

In [4]:
from zuko.transforms import ComposedTransform

num_poses = 10
num_sols = 1000
Q, P = nsf.robot.sample_joint_angles_and_poses(n=num_poses)

# P.shape = (100, 7)
# P = torch.from_numpy(P).to('cuda')
# s = torch.ones(P.shape[0], 1).to('cuda')
# c = torch.column_stack([P, s])
# c_base = base(c)
# c_transforms = ComposedTransform(*[t(c) for t in transforms])

In [5]:
z = np.random.randn(num_sols, num_poses, nsf.n) * 0.25
J_hat = nsf.generate_ik_solutions_z(P, z)

[INFO] zsample: torch.Size([1000, 10, 7])


In [6]:
l2, ang = evaluate_pose_error_J3d_P2d(
    # input J.shape = (num_sols, num_poses, num_dofs or n)
    nsf.robot, J_hat, P, return_all=True
)
l2.mean(), ang.mean()

(0.0027180834984647, 0.02797915218292401)

In [ ]:
conditions_torch = c.to('cuda')
conditions_torch = conditions_torch.to(torch.float32)
z = torch.randn((num_sols, num_poses, nsf.n), dtype=torch.float32).to('cuda') * 0.25
J = nsf._solver(conditions_torch).zsample(z)

In [ ]:
J.shape

In [ ]:
c_transforms.inv(z)

In [ ]:
from zuko.transforms import ComposedTransform
from zuko.flows.core import LazyDistribution
from torch.distributions.distribution import Distribution

class CustomTransform(Distribution):
    def __init__(self, transforms, base):
        self.transforms = transforms
        self.base = base

    def rsample(self, sample_shape=torch.Size()):
        z = self.base.sample(sample_shape)
        return self.transforms.inv(z)

class CustomFlow(LazyDistribution):
    def __init__(self, transforms, base):
        super().__init__()
        self.transforms = transforms
        self.base = base
        
    def forward(self, c):
        base = self.base(c)
        transforms = ComposedTransform(*(t(c) for t in self.transforms))
        return CustomTransform(transforms, base)
    
def compute_transforms(c, z):
    c_transforms = ComposedTransform(*[t(c) for t in transforms])
    return c_transforms.inv(z)
